In [ ]:
import datetime
import os
import glob
import numpy as np
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap

clon=180

land_10m = cfeature.NaturalEarthFeature(
    "physical", "land", "10m", facecolor=cfeature.COLORS["land"]
)
    
def plot_decor(ax):
    ax.coastlines("10m")
    ax.set_extent([172,180,-42,-34])
    ax.add_feature(land_10m, zorder=-1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    

def plot_1x1(da, levels, cmap, cbar_label, fig_label):
    fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
    da.plot.contourf(ax=ax, levels=levels, cmap=cmap, cbar_kwargs={'label': cbar_label}, transform=ccrs.PlateCarree())                                            
    plot_decor(ax)
    fig.suptitle(fig_label)
    return fig,ax



# Set customizable global variables

In [ ]:
density = 1300 # kg/m3 andesite for Tongariro
it = -1  # plots for last time instant in files (same as it=23)

savefigs = True
fig_args = dict(bbox_inches='tight', dpi=300)

# UNCOMMENT FOR OPERATIONAL CASE STUDY
data_pattern = '202107150000/output/*Tongariro*nc'
fileplot_prefix = 'plt_ens_202107150000_Tongariro'

# UNCOMMENT FOR HISTORICAL CASE STUDY
# data_pattern = '201208061200/output/*Tongariro*nc'
# fileplot_prefix = 'plt_ens_201208061200_Tongariro'

# Read files

In [ ]:
files = glob.glob(data_pattern)
files.sort()
print(len(files))

for x in files[:3]:
    print(x)

In [ ]:
ds = xr.open_mfdataset(files, combine='nested', concat_dim=['idx'])
ds

In [ ]:
eruption_time = datetime.datetime.strptime(ds.attrs['eruption_time'], '%Y-%m-%dT%H:%M:%S')
print(eruption_time)
lag_hours = np.array([int(x.total_seconds()/3600.) for x in pd.to_datetime(ds.time.data) - eruption_time])
print(lag_hours)

# Compute exceedance probabilities

In [ ]:
def compute_arrival_time(dain, vmin):
    if dain.dims[0] != 'time':
        print(dain.shape)
        print('bad shape. Shoud be time, lat, lon')
        raise
    nt = dain.sizes['time']
    daout = xr.where(dain > vmin, 1., 0)
    daout = daout.cumsum(dim='time')
    daout = daout.where(daout > 0)
    # Assuming hourly data
    for it in range(nt):
         daout.data[it,...] = it+2 - daout.isel(time=it).data
    return daout

def compute_exc_prob_arrival_time(dain, threshold, arrival_min):
    da = xr.where(dain > threshold, 1, 0)
    da_prob = da.sum(dim='idx')/da.sizes['idx']
    da_prob_arrival_time = compute_arrival_time(da_prob, vmin=arrival_min)
    return(da_prob, da_prob_arrival_time)


In [ ]:
thresholds = [0.01, 0.1, 1, 3, 10]
arrival_min = 0.5  # arrival of the 50% excedance prob

da_probs = []
da_prob_arrival_times = []
for thres in thresholds:
    da_prob, da_prob_arrival_time = compute_exc_prob_arrival_time(
        dain=ds['total_deposition'],
        threshold=thres,    
        arrival_min=arrival_min)
    da_probs += [da_prob]
    da_prob_arrival_times += [da_prob_arrival_time]

da_probs = xr.concat(da_probs, pd.Index(thresholds, name="threshold"))
da_prob_arrival_times = xr.concat(da_prob_arrival_times, pd.Index(thresholds, name="threshold"))

# Plot exceedance probability and arrival time

In [ ]:
tic = datetime.datetime.now()

prob_levs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# prob_levs = [0.1, 0.5, 0.9, 1.]

for thres in thresholds[:1]:

    da_prob = da_probs.sel(threshold=thres)
    da_prob_arrival_time = da_prob_arrival_times.sel(threshold=thres)

    # plot arrival time
    fig, ax = plot_1x1(
        da = da_prob_arrival_time.isel(time=it),
        levels = lag_hours,
        cmap = 'jet_r',
        cbar_label = 'Arrival time of {}% probability of ash thickness > {} mm [h]'.format(int(arrival_min*100), thres),
        fig_label = 'Arrival time ({} h after eruption)'.format(lag_hours[it]),
    )
    fileplot = fileplot_prefix + '_excprob{}_arrivaltime_lag{:02d}.png'.format(thres, lag_hours[it])
    print(fileplot)
    if savefigs: fig.savefig(fileplot, **fig_args)    
    
    
    # plot excprob 
    da = da_prob.isel(time=it)
    da = da.where(da > prob_levs[0])    
    fig, ax = plot_1x1(
        da = da,
        levels = prob_levs,
        cmap = 'viridis',
        cbar_label = 'Probability of ash thickness > {} mm [-]'.format(thres),
        fig_label = 'Excedance probability of {} mm ({} h after eruption)'.format(thres, lag_hours[it]),
    )   
    da.plot.contour(ax=ax, levels=[0.5], colors=['red'],  transform=ccrs.PlateCarree())
    fileplot = fileplot_prefix + '_excprob{}_lag{:02d}.png'.format(thres, lag_hours[it])
    print(fileplot)
    if savefigs: fig.savefig(fileplot, **fig_args)    

    toc = (datetime.datetime.now() - tic).total_seconds()
    print('Elapsed time: {} s'.format(toc))
    

## Plot Hazard Matrix

In [ ]:
ashbins = [0.01, 0.1, 1, 10]
probbins = [0.1, 0.5, 0.9, 1.]

# yellow, orange, red
risk = [1,2,3]
cmap_risk = ListedColormap(["yellow", "orange", "red"])

it = -1
da_probs_it = da_probs.isel(time=it)

In [ ]:
# yellow: everything > probbins[0]
da_risk_it = xr.where(da_probs_it.sel(threshold=ashbins[0]) > probbins[0], 1, 0)

In [ ]:
# orange:
tmp1 = xr.where(da_probs_it.sel(threshold=ashbins[1]) > probbins[1], 1, 0)
tmp2 = xr.where(da_probs_it.sel(threshold=ashbins[2]) > probbins[0], 1, 0)
tmp = tmp1 + tmp2
tmp = xr.where(tmp > 0, 1, 0)
da_risk_it = xr.where(tmp > 0, 2, da_risk_it)

In [ ]:
# red
tmp3 = xr.where(da_probs_it.sel(threshold=ashbins[2]) > probbins[1], 1, 0)
tmp4 = xr.where(da_probs_it.sel(threshold=ashbins[1]) > probbins[2], 1, 0)
tmp = tmp3 + tmp4
tmp = xr.where(tmp > 0, 1, 0)
da_risk_it = xr.where(tmp > 0, 3, da_risk_it)

In [ ]:
da_risk_it = da_risk_it.drop('threshold')

In [ ]:
def plot_inset_matrix(iax, ashbins, probbins, cmap):
    Z = np.array([[1, 1, 2],
                  [1, 2, 3],
                  [1, 3, 3]])
    x = np.arange(Z.shape[1] + 1)
    y = np.arange(Z.shape[0] + 1)
    iax.pcolormesh(x, y, Z, shading='flat', vmin=Z.min(), vmax=Z.max(), cmap=cmap,
                  edgecolor='k')
    iax.set_xticks([0, 1., 2., 3.])
    iax.set_xticklabels(ashbins)
    iax.set_xlabel('Ash deposition [mm]')
    iax.set_yticks([0, 1., 2., 3.])
    iax.set_yticklabels(np.array(probbins)*100.)
    iax.set_ylabel('Likelihood [\%]')
    iax.set(title='Hazard matrix')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
da = da_risk_it.where(da_risk_it > 0) 
da.plot.contourf(ax=ax, 
                 levels=[0,1,2,3], cmap=cmap_risk,
                 add_colorbar=False,
                 transform=ccrs.PlateCarree())   

ds['total_deposition'].isel(idx=0, time=-1).plot.contour(ax=ax, levels=[0.5], colors=['blue'],  transform=ccrs.PlateCarree())

plot_decor(ax)
ax.set_extent([170,180,-42,-34])
fig.suptitle('Likelihood of accumulated ash thickness ({} h after eruption)'.format(lag_hours[it]))

# this is an inset axes over the main axes
ins_ax = fig.add_axes([.26, .5, .12, .15])
plot_inset_matrix(ins_ax, ashbins, probbins, cmap_risk)

fileplot = fileplot_prefix + '_hazard_matrix_lag{:02d}.png'.format(lag_hours[it])
print(fileplot)
if savefigs: fig.savefig(fileplot, **fig_args)  
    